In [68]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "dirty_cafe_sales.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training",
  file_path
)

C:\Users\PC\AppData\Local\Temp\ipykernel_15284\1585058692.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


In [69]:
 df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [57]:
# Standardize column names.
# Remove leading/trailing spaces, replace spaces with underscores, and convert to lowercase.
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10000 non-null  object
 1   item              9667 non-null   object
 2   quantity          9862 non-null   object
 3   price_per_unit    9821 non-null   object
 4   total_spent       9827 non-null   object
 5   payment_method    7421 non-null   object
 6   location          6735 non-null   object
 7   transaction_date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [41]:
# We will investigate column by column to identify some data consistency issues.
# Let check number of unique value in each column first to have some insights.

df.nunique()

transaction_id      10000
item                   10
quantity                7
price_per_unit          8
total_spent            19
payment_method          5
location                4
transaction_date      367
dtype: int64

In [44]:
# First column, transaction_id is unique for all records, so no issues there.
# Next, let's check item column value counts to see if there are any inconsistencies.

df['item'].value_counts()

item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
unknown      636
Name: count, dtype: int64

In [ ]:
# In item column, we have two values representing unknown items: 'UNKNOWN' and 'ERROR'.
# We will standardize these to a single value 'unknown'.
df['item'] = df['item'].replace(['UNKNOWN','ERROR'], 'unknown')

In [61]:
df['quantity'].value_counts()

quantity
5          2013
2          1974
4          1863
3          1849
1          1822
unknown     341
Name: count, dtype: int64

In [62]:
df['price_per_unit'].value_counts()

price_per_unit
3.0        2429
4.0        2331
2.0        1227
5.0        1204
1.0        1143
1.5        1133
unknown     354
Name: count, dtype: int64

In [63]:
df['total_spent'].value_counts()

total_spent
6.0        979
12.0       939
3.0        930
4.0        923
20.0       746
15.0       734
8.0        677
10.0       524
2.0        497
9.0        479
5.0        468
16.0       444
unknown    329
25.0       259
7.5        237
1.0        232
4.5        225
1.5        205
Name: count, dtype: int64

In [64]:
df['payment_method'].value_counts()

payment_method
Digital Wallet    2291
Credit Card       2273
Cash              2258
unknown            599
Name: count, dtype: int64

In [65]:
df['location'].value_counts()

location
Takeaway    3022
In-store    3017
unknown      696
Name: count, dtype: int64

In [66]:
df['transaction_date'].value_counts()

transaction_date
unknown       301
2023-06-16     40
2023-02-06     40
2023-09-21     39
2023-07-21     39
             ... 
2023-11-24     15
2023-04-27     15
2023-07-22     14
2023-02-17     14
2023-03-11     14
Name: count, Length: 366, dtype: int64

In [ ]:
# 1. Select only the columns that have the issue
cols_to_fix = ['item', 'quantity','price_per_unit','total_spent','payment_method','location','transaction_date']

# 2. Define replacement rules mapping. In this case we want to replace both 'ERROR' and 'UNKNOWN' with 'unknown'
rules = {
    'ERROR': 'unknown',
    'UNKNOWN': 'unknown'
}

# 3. Apply the fix safely
df[cols_to_fix] = df[cols_to_fix].replace(rules)

In [67]:
# This will show you the new counts for your cleaned columns
for col in cols_to_fix:
    print(f"--- {col} counts ---")
    print(df[col].value_counts())
    print("\n")

--- item counts ---
item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
unknown      636
Name: count, dtype: int64


--- quantity counts ---
quantity
5          2013
2          1974
4          1863
3          1849
1          1822
unknown     341
Name: count, dtype: int64


--- price_per_unit counts ---
price_per_unit
3.0        2429
4.0        2331
2.0        1227
5.0        1204
1.0        1143
1.5        1133
unknown     354
Name: count, dtype: int64


--- total_spent counts ---
total_spent
6.0        979
12.0       939
3.0        930
4.0        923
20.0       746
15.0       734
8.0        677
10.0       524
2.0        497
9.0        479
5.0        468
16.0       444
unknown    329
25.0       259
7.5        237
1.0        232
4.5        225
1.5        205
Name: count, dtype: int64


--- payment_method counts ---
payment_method
Digital Wallet    2291
Credit Card       2273
Cash              2258
u